# **Hackathon 2 Team 1 Project**

# Section 1

## Objectives

In this section, the aim is to prepare a cleaned dataset for visualization and analysis from the raw data files. There are ETL procedures form data extraction, data cleaning and processing to data load.


## Inputs

* Datasets used for this analysis is the retail data set from Kaggle (https://www.kaggle.com/datasets/manjeetsingh/retaildataset). 

* 1 raw file will be used.
    *[insurance.csv](../dataset/raw/insurance.csv) 

## Outputs

* A cleaned dataset will be save as a CSV file below
    *  [insurance_cleaned.csv](../dataset/processed/insurance_cleaned.csv)





---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/Users/denniskwok/Documents/data-analytics/hackathon2-team1-pilot/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/Users/denniskwok/Documents/data-analytics/hackathon2-team1-pilot'

---

# Part A

# Data Extraction

In [28]:
import numpy as np
import pandas as pd

* **Step 1: Load Dataset**

In [29]:
# Load dataset from csv file
def load_csv(filepath):
    try:
        df = pd.read_csv(filepath)
        print(f"Loaded {filepath} successfully.")
        return df
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

df_insurance = load_csv("dataset/raw/insurance.csv")


Loaded dataset/raw/insurance.csv successfully.


* **Step 2: Overview The RAW Dataset**

**General dataframe information** 

In [30]:
df_insurance.info() # Display dataframe information  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


**Dataframe data overview**

In [31]:
df_insurance.head() # Display the first few rows of the dataframe

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


**Checking missing values**

In [32]:
df_insurance.isnull().sum() # Check for missing values

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

---

# Part B

# Data Transformation (data cleaning and processing)

* **Step 1: Create Pipeline for Transformation**

     * i. Drop Missing Data
     * ii. Convert Categorical Columns to Numerical
     * iii. Scale the Continuous Numerical Columns

In [10]:
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler
from feature_engine.encoding import OneHotEncoder
from feature_engine.imputation import DropMissingData
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('drop_missing', DropMissingData()),
    ('onehot_encoder', OneHotEncoder(variables=['sex', 'smoker', 'region'], drop_last=False)),
    ('std_scaler', SklearnTransformerWrapper(
        transformer=StandardScaler(),
        variables=['age', 'bmi', 'charges']))
])

df_insurance_transformed = pipeline.fit_transform(df_insurance)

**Check the OneHotEncoder Mapping**

In [27]:
# Access the fitted OneHotEncoder from the pipeline
encoder_step = pipeline.named_steps['onehot_encoder']


# Show the mapping dictionary
print('Mapping Dictionary:\n')
print(encoder_step.encoder_dict_, '\n\n')

print('New Columns after One-Hot Encoding:\n')

#new columns after the full pipeline transformation
print(df_insurance_transformed.columns.tolist(), '\n\n')

Mapping Dictionary:

{'sex': ['female', 'male'], 'smoker': ['yes', 'no'], 'region': ['southwest', 'southeast', 'northwest', 'northeast']} 


New Columns after One-Hot Encoding:

['age', 'bmi', 'children', 'charges', 'sex_female', 'sex_male', 'smoker_yes', 'smoker_no', 'region_southwest', 'region_southeast', 'region_northwest', 'region_northeast'] 




* **Step 2: Final Check for Missing Values after Feature-engine Transformation**

In [ ]:
# Check missing values in the insurance DataFrame after transformation
df_insurance_transformed.isnull().sum()

age                 0
bmi                 0
children            0
charges             0
sex_female          0
sex_male            0
smoker_yes          0
smoker_no           0
region_southwest    0
region_southeast    0
region_northwest    0
region_northeast    0
dtype: int64

There is no missing data in the dataset.

* **Step 3: Overview and Inspect the data after Feature-Engineering Transformation**

In [13]:
# Overview head and info for final transformed dataframe
display(df_insurance_transformed.head())
print(df_insurance_transformed.columns.tolist())
df_insurance_transformed.info()

,age,bmi,children,charges,sex_female,sex_male,smoker_yes,smoker_no,region_southwest,region_southeast,region_northwest,region_northeast
0,-1.438764,-0.453320,0,0.298584,1,0,1,0,1,0,0,0
1,-1.509965,0.509621,1,-0.953689,0,1,0,1,0,1,0,0
2,-0.797954,0.383307,3,-0.728675,0,1,0,1,0,1,0,0
3,-0.441948,-1.305531,0,0.719843,0,1,0,1,0,0,1,0
4,-0.513149,-0.292556,0,-0.776802,0,1,0,1,0,0,1,0


['age', 'bmi', 'children', 'charges', 'sex_female', 'sex_male', 'smoker_yes', 'smoker_no', 'region_southwest', 'region_southeast', 'region_northwest', 'region_northeast']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   float64
 1   bmi               1338 non-null   float64
 2   children          1338 non-null   int64  
 3   charges           1338 non-null   float64
 4   sex_female        1338 non-null   int64  
 5   sex_male          1338 non-null   int64  
 6   smoker_yes        1338 non-null   int64  
 7   smoker_no         1338 non-null   int64  
 8   region_southwest  1338 non-null   int64  
 9   region_southeast  1338 non-null   int64  
 10  region_northwest  1338 non-null   int64  
 11  region_northeast  1338 non-null   int64  
dtypes: float64(3), int64(9)
memory usage: 125.6 KB


---

# Part C

# Data Load (creating a cleaned dataset)

After review and processing, 2 datasets had been prepared. The original dataset Insurance_clearned_orig.csv is mainly for general visualization while the transformed dataset Insurance_cleaned_transformed is prepared for further Regression and Predictive model application.

In [26]:
# Export the cleaned dataframe to a CSV file in the folder for processed CSV
df_cleaned_orig = df_insurance.copy()
df_cleaned_orig.to_csv('dataset/processed/Insurance_cleaned_orig.csv', index=False)
df_cleaned_transformed = df_insurance.copy()
df_cleaned_transformed.to_csv('dataset/processed/Insurance_cleaned_transformed.csv', index=False)

print(f'Two cleaned dataset exported to dataset/processed/')

Two cleaned dataset exported to dataset/processed/


---

To be contined in Section 2 Data Visualization.